In [1]:
import os.path
import scipy as sp
from scipy.linalg import block_diag
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
import sys as sys
import time
import matplotlib.pyplot as plt

This is where you set the inputs. `restrictType` and `prolongType` sets the restriction-prolongation pair of methods for which you want to observe the grid-transfer on an operator. For injection, set `restrictType` to `'I'`. For full-weighting, set `restrictType` to `'FW'`. For piecewise interpolation, set `prolongType` to `'PW'`. For linear interpolation, set `prolongType` to `'LI'`.

In [2]:
# Touching this is fine.
dim = 1
nh = 4
restrictType = 'FW'
prolongType = 'LI'

# Don't touch this.
x = np.linspace(0, 1, num = nh)
y = 1 * x
z = 1 * y
oneSpace = np.ones(nh, float)

# Touching this is fine.
heatCoeff = Sine(nh, x, '+', y)
print(heatCoeff)

NameError: name 'Sine' is not defined

This block checks to make sure your inputs aren't garbage.

In [ ]:
check = nh
while (check % 2 == 0):
    check = check / 2
if (check != 1):
    sys.exit('ERROR:\nnh must be a base-2 integer!')

This function makes sure that dimensionalities match up appropriately.

In [ ]:
def CheckDim(nh, checkMatrix):
    dim = size(shape(checkMatrix))
    problem = 0
    for i in range(dim):
        if (nh != shape(checkMatrix)[i]):
            problem = problem + 1
            print(i, problem)
    return problem

In [ ]:
def Operate(a, operation, b):
    if (operation == '+'):
        c = a + b
    else:
        if (operation == '-'):
            c = a - b
        else:
            if (operation == '*'):
                c = np.outer(a, b)
            else:
                if (operation == '/'):
                    c = np.asarray([a[i] / b[i] for i in a])
                else:
                    if ((operation == '^') or (operation == '**')):
                        c = np.asarray([a[i] / b[i] for i in a])
                    else:
                        sys.exit('ERROR:\nOperate:\nInvalid operation input!')
    return c

In [ ]:
def Sine(nh, x, operation1 = '+', y = np.ones(nh, float), operation2 = '+', z = np.ones(nh, float)):
    xSpace, ySpace, zSpace = np.meshgrid(x, y, z)
    fxGrid = np.sin(Operate(Operate(xSpace, operation1, ySpace), operation2, zSpace))
    print(fxGrid)
    fx = fxGrid.reshape(nh ** 3)
    return fx

This function changes the dimensionality of square operators. For 1-D cases, its output is the input matrix. For 2-D cases, its output is a block-tridiagonal matrix wherein the main diagonal comprises the input matrix and the sub- and super-diagonal comprise identity matrices. For 3-D cases, its output is a block-tridiagonal wherein the main diagonal comprises the 2-D matrices and the sub- and super-diagonal again comprise identity matrices.

In [ ]:
def FixDim(nh, blockMatrix, dim):
    problem = CheckDim(nh, blockMatrix)
    if (problem != 0):
        sys.exit('ERROR:\nFixDimensionality:\nnh - 1 does not match dimensionality of blockMatrix!')
    h = 1.0 / nh
    if (dim == 1):
        outputMatrix = blockMatrix
    else:
        print('WARNING:')
        print('This program is only in the preliminary stages of handling dimensionality higher than 1! Errors are')
        print('likely to occur!')
        blockMatrices2D = [blockMatrix for x in range(nh - 1)]
        outputMatrix2D = block_diag(*blockMatrices2D)
        fill_diagonal(outputMatrix2D[nh - 1:], -1.0 / (h ** 2))
        fill_diagonal(outputMatrix2D[:, nh - 1:], -1.0 / (h ** 2))
        fill_diagonal(outputMatrix2D, 2 * outputMatrix2D[0, 0])
        if (dim == 2):
            outputMatrix = outputMatrix2D
        else:
            if (dim == 3):
                blockMatrices3D = [outputMatrix2D for x in range(nh - 1)]
                outputMatrix = block_diag(*blockMatrices3D)
                fill_diagonal(outputMatrix[(nh - 1) ** 2:], -1.0 / (h ** 2))
                fill_diagonal(outputMatrix[:, (nh - 1) ** 2:], -1.0 / (h ** 2))
                fill_diagonal(outputMatrix, 1.5 * outputMatrix[0, 0])
            else:
                sys.exit('ERROR:\nMakeLaplacian:\nDimensionality cannot be greater than 3!')
    return outputMatrix

This function manages the construction of the Laplacian operator.

In [ ]:
def MakeLaplacian(nh, dim):
    Laplacian1D = MakeLaplacian1D(nh)
    Laplacian = FixDim(nh, Laplacian1D, dim)
    return Laplacian

This function constructs a 1-D Laplacian operator.

In [ ]:
def MakeLaplacian1D(nh):
    h = 1.0 / nh
    Laplacian = np.zeros((nh, nh), float)
    fill_diagonal(Laplacian[1:], -1)
    fill_diagonal(Laplacian[:, 1:], -1)
    fill_diagonal(Laplacian, 2)
    Laplacian = (1.0 / (h ** 2)) * Laplacian
    return Laplacian

This function manages the construction of the Helmholtz operator.

In [ ]:
def MakeHelmholtz(nh, heatCoeff, dim):
    problem = CheckDim(nh, heatCoeff)
    if (problem != 0):
        sys.exit('ERROR:\nMakeHelmholtz:\nnh - 1 does not match dimensionality of heatCoeff!')
    delOp = np.zeros((nh, nh), float)
    fill_diagonal(delOp[1:], 1)
    fill_diagonal(delOp[:, 1:], -1)
    Helmholtz1D = np.matmul(np.matmul(delOp, heatCoeff), delOp)
    Helmholtz = FixDim(nh, Helmholtz1D, dim)
    return Helmholtz

This function creates an injection operator. Although `dim` is preemptively being passed into it, it is not yet set up to handle dimensionality greater than 1.

In [ ]:
def MakeInject(nh, dim):
    n2h = int(nh / 2)
    inject = np.zeros((n2h, nh), float)
    for i in range(n2h):
        inject[i, (2 * i) + 1] = 1
    return inject

This function creates a full-weighting operator. Although `dim` is preemptively being passed into it, it is not yet set up to handle dimensionality greater than 1.

In [ ]:
def MakeFullWeight(nh, dim):
    n2h = int(nh / 2)
    fullWeight = np.zeros((n2h, nh), float)
    weights = [0.5, 0.5]
    for i in range(n2h):
        fullWeight[i, (2 * i):(2 * i) + 2] = weights
    return fullWeight

This function creates a piecewise interpolation operator. Although `dim` is preemptively being passed into it, it is not yet set up to handle dimensionality greater than 1.

In [ ]:
def MakePiecewise(nh, dim):
    nh2 = 2 * nh
    piecewise = np.zeros((nh2, nh), float)
    weights = [1, 1]
    for i in range(nh):
        piecewise[(2 * i):(2 * i) + 2, i] = weights
    return piecewise

This function creates a linear interpolation operator. Although `dim` is preemptively being passed into it, it is not yet set up to handle dimensionality greater than 1.

In [ ]:
def MakeLinearInterp(nh, dim):
    nh2 = 2 * nh
    linearInterp = np.zeros((nh2, nh), float)
    weights = [1, 1]
    for i in range(nh):
        linearInterp[(2 * i):(2 * i) + 2, i] = weights
    return linearInterp

This function restricts some arbitrary operator from $h$ to 2$h$ using whichever restriction and prolongation combination the user prefers. Although `dim` is preemptively being passed into it, it is not yet set up to handle dimensionality greater than 1.

In [ ]:
def RestrictOperator(nh, operatorh, restrictType, prolongType, dim):
    problem = CheckDim(nh, operatorh)
    if (problem != 0):
        sys.exit('ERROR:\nRestrictOperator:\nnh - 1 does not match dimensionality of Laplacian!')
    n2h = int(nh / 2)
    if (restrictType == 'I'):
        restrict = MakeInject(nh, dim)
    else:
        restrict = MakeFullWeight(nh, dim)
    if (prolongType == 'PW'):
        prolong = MakePiecewise(n2h, dim)
    else:
        prolong = MakeLinearInterp(n2h, dim)
    operator2h = np.matmul(restrict, np.matmul(operatorh, prolong))
    return Laplacian2h

This function prolongs some arbitrary operator from 2$h$ to $h$ using whichever restriction and prolongation combination the user prefers. Although `dim` is preemptively being passed into it, it is not yet set up to handle dimensionality greater than 1.

In [ ]:
def ProlongOperator(nh, operator2h, restrictType, prolongType, dim):
    problem = CheckDim(nh, operator2h)
    if (problem != 0):
        sys.exit('ERROR:\nProlongOperator:\nnh - 1 does not match dimensionality of Laplacian!')
    nh2 = 2 * nh
    if (restrictType == 'I'):
        restrict = MakeInject(nh2, dim)
    else:
        restrict = MakeFullWeight(nh2, dim)
    if (prolongType == 'PW'):
        prolong = MakePiecewise(nh, dim)
    else:
        prolong = MakeLinearInterp(nh, dim)
    operatorh = np.matmul(prolong, np.matmul(operator2h, restrict))
    return operatorh

This block is just a workspace with useful things to streamline testing as I develop new pieces of code.

In [ ]:
n2h = int(nh / 2)
nh2 = 2 * nh

test = np.arange((nh2 - 1) * (nh - 1)).reshape((nh2 - 1, nh - 1))

This block creates a Laplacian operator and a Helmholtz operator using the conditions you set in the input block. It also creates examples of restriction and prolongation operators.

In [ ]:
# make Laplacian operator
Laplacian = MakeLaplacian(nh, dim)

# make Helmholtz operator
Helmholtz = MakeHelmholtz(nh, heatCoeff, dim)

# make restriction operators
inject = MakeInject(nh, dim)    
fullWeight = MakeFullWeight(nh, dim)

# make prolongation operators
piecewise = MakePiecewise(n2h, dim)
linearInterp = MakeLinearInterp(n2h, dim)

This block prints out the operators created in the block above.

In [ ]:
np.set_printoptions(threshold = sys.maxsize)
print('Laplacian operator =')
print(Laplacian)
print('')
print('Helmholtz operator =')
print(Helmholtz)
print('')
print('injection operator =')
print(inject)
print('')
print('full weighting operator =')
print(fullWeight)
print('')
print('piecewise interpolation operator =')
print(piecewise)
print('')
print('linear interpolation operator =')
print(linearInterp)

This block creates Laplacian operators and a Helmholtz operators with different grid spacing for comparing the explicitly-created and the restricted and prolongated versions of them, using the conditions for restriction and prolongation you set in the input block.

In [ ]:
Laplacian2h = MakeLaplacian(n2h, dim)
Laplacian2hRestricted = RestrictOperator(nh, Laplacian, restrictType, prolongType, dim)

Laplacianh2 = MakeLaplacian(nh2, dim)
Laplacianh2Prolongated = ProlongOperator(nh, Laplacian, restrictType, prolongType, dim)

xSpace = np.linspace(0, 1, num = n2h)
heatCoeff = np.sin(xSpace)

Helmholtz2h = MakeHelmholtz(n2h, heatCoeff, dim)
Helmholtz2hRestricted = RestrictOperator(nh, Helmholtz, restrictType, prolongType, dim)

xSpace = np.linspace(0, 1, num = nh2)
heatCoeff = np.sin(xSpace)

Helmholtzh2 = MakeHelmholtz(nh2, heatCoeff, dim)
Helmholtzh2Prolongated = ProlongOperator(nh, Helmholtz, restrictType, prolongType, dim)

This block prints out the operators created in the block above.

In [ ]:
print('explicitly-calculated Laplacian^2h =')
print(Laplacian2h)
print('')
print('restricted Laplacian^2h =')
print(Laplacian2hRestricted)
print('')
print('explicitly-calculated Laplacian^h =')
print(Laplacianh2)
print('')
print('prolongated Laplacian^h =')
print(Laplacianh2Prolongated)
print('')
print('explicitly-calculated Helmholtz^2h =')
print(Helmholtz2h)
print('')
print('restricted Helmholtz^2h =')
print(Helmholtz2hRestricted)
print('')
print('explicitly-calculated Helmholtz^h =')
print(Helmholtzh2)
print('')
print('prolongated Helmholtz^h =')
print(Helmholtzh2Prolongated)